# Homework 3: Twitter POS tagging

Student Name:Mao Yundong

Student ID:882542

Python version used:2.7.14

## General info

<b>Due date</b>: 11pm, Sunday April 15

<b>Submission method</b>: see LMS

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -20% per day

<b>Marks</b>: 5% of mark for class

<b>Overview</b>: In this homework, you'll be adapting a POS tagger to Twitter data, starting from a tagger trained on Penn Treebank. You will also use prior information on the Twitter tagset to obtain better performance. Finally, you will also analyse your results in a more fine-grained way. For extra credits, you will implement the Expectation-Maximisation algorithm.

<b>Materials</b>: See the main class LMS page for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages; if your iPython notebook doesn't run on the marker's machine, you will lose marks.  

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Extra credit</b>: Each homework has a task which is optional with respect to getting full marks on the assignment, but that can be used to offset any points lost on this or any other homework assignment (but not the final project or the exam). We recommend you skip over this step on your first pass, and come back if you have time: the amount of effort required to receive full marks (1 point) on an extra credit question will be substantially more than earning the same amount of credit on other parts of the homework.

<b>Updates</b>: Any major changes to the assignment will be announced via LMS. Minor changes and clarifications will be announced in the forum on LMS, we recommend you check the forum regularly.

<b>Academic Misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.


### Part 1: Preprocessing (2.0)

<b>Instructions</b>: you first task is to preprocess the data. We will use two datasets for training: 1) the Penn Treebank sample used in the workshops and 2) the Twitter samples data you used in Homework 1. In order to adapt the tagger to the Twitter data we need to built a *joint* vocabulary containing all the word types in PTB and the twitter_samples corpora. So, in addition to preprocessing, your code should also build this vocabulary. Finally, you should also store the tagset, for reasons that will be clearer later.

<b>Important</b>: you are allowed to reuse all the code from the workshop notebooks. In fact you are encouraged to do so as much of this homework will be based on these notebooks.

The vocabulary and the tagset should be stored in Python dictionaries, mapping each word (or tag) to an index (integer). This is similar to what is done in the W6/W7 workshop notebooks. The preprocessed corpora should contain indices only, as in the workshop.

Let's start with the PTB data. You should iterate over all sentences and words, and build the vocabulary and the tagset. Important: make sure you <b>lowercase</b> words before they are added to the dictionary. You should also generate the preprocessed corpus. It should be a list where each element is a tagged sentence, represented as another list of (word, tag) indices (which should correspond to the original words/tags). Print the first preprocessed sentence, the index for the word 'electricity' and the length of the full tagset. (0.5)


In [43]:
from nltk.corpus import treebank
word_numbers = {}
tag_numbers = {}
corpus = treebank.tagged_sents()
num_corpus = []
for sent in corpus:
    num_sent = []
    for word, tag in sent:
        wi = word_numbers.setdefault(word.lower(), len(word_numbers))
        ti = tag_numbers.setdefault(tag, len(tag_numbers))
        num_sent.append((wi, ti))
    num_corpus.append(num_sent)
print num_corpus[0]
print word_numbers["electricity"]
# print len(word_numbers)
print len(tag_numbers)
# print len(word_numbers)


[(0, 0), (1, 0), (2, 1), (3, 2), (4, 3), (5, 4), (2, 1), (6, 5), (7, 6), (8, 7), (9, 8), (10, 9), (11, 7), (12, 4), (13, 8), (14, 0), (15, 2), (16, 10)]
1095
46


<b>Instructions</b>: now you should do the same with the twitter_samples dataset. From now on, we will refer this dataset as the **training** tweets. Since this data is not tagged, the preprocessed corpus should be a list where each element is another list containing indices only (instead of (word, tag) tuples). A tokenised version of twitter_samples is available through the method .tokenized(), use this method to read your corpus. Besides generating the corpus, you should also **update** the vocabulary with the new words from this corpus.

There are two things to keep in mind when doing this process:

1) We will perform a bit more of preprocessing in this dataset, besides lowercasing. Specifically, you should replace special tokens with special symbols, as follows:
- Username mentions are tokens that start with '@': replace these tokens with 'USER_TOKEN'
- Hashtags are tokens that start with '#': replace these with 'HASHTAG_TOKEN'
- Retweets are represented as the token 'RT' (or 'rt' if you lowercase first): replace these with 'RETWEET_TOKEN'
- URLs are tokens that start with 'https://' or 'http://': replace these with 'URL_TOKEN'

2) **Do not create a new vocabulary**. Instead, you should update the vocabulary built from PTB with any new words present in this corpus. These should *include* the special tokens defined above but *not* the original un-preprocessed tokens.

The easiest way to do these steps is by doing 3 passes over the data: preprocess the words first, update the vocabulary and finally convert the corpus into the list format described above. However, it is possible to do all of this in one pass only.

Print the first sentence from your preprocessed corpora, the index for the word 'electricity' and the index for 'HASHTAG_TOKEN'. (0.5)

In [44]:
import nltk
from nltk.corpus import twitter_samples
twitter_corpus = []
tokens_list = twitter_samples.tokenized()
# print tokens[0]

for tokens in tokens_list:
    temp = []
    for token in tokens:
        if token[0] =="@":
            temptoken = "USER_TOKEN"
        elif token[0] == "#":
            temptoken = "HASHTAG_TOKEN"
        elif token == "RT":
            temptoken = "RETWEET_TOKEN"
        elif token[:min(len("https://"),len(token))] == "https://":
            temptoken = "URL_TOKEN"
        elif token[:min(len("http://"),len(token))] == "http://":
            temptoken = "URL_TOKEN"
        else:
            temptoken = token.lower()
        wi = word_numbers.setdefault(temptoken,len(word_numbers))
        temp.append(wi)
    twitter_corpus.append(temp)

print twitter_corpus[0]
print word_numbers["electricity"]
print word_numbers["HASHTAG_TOKEN"]
    


[11387, 182, 11388, 11389]
1095
11409


<b>Instructions:</b> now we will preprocess the tagged twitter corpus used in W7 (Ritter et al.). This dataset will be referred from now on as **test** tweets. Before you do that though, you should update the tagset.

You might have noticed this in the workshop but this dataset has a few extra tags, besides the PTB ones. These were added to incorporate specific phenomena that happens on Twitter:
- "USR": username mentions
- "HT": hashtags
- "RT": retweets
- "URL": URL addresses

Notice that these special tags correspond to the special tokens we preprocessed before. These steps will be important in Part 3 later.

There a few additional tags which are not specific to Twitter but are not present in the PTB sample:
- "VPP"
- "TD"
- "O"

You should add these new seven tags to the tagset you built when reading the PTB corpus.

Another task is to add an extra type to the vocabulary: `<unk>`. This is in order to account for unknown or out-of-vocabulary words.

Finally, build two "inverted indices" for the vocabulary and the tagset. These should be lists, where the "i"-th element should contain the word (or tag) corresponding to the index "i" in the vocabulary (or tagset).

After doing these tasks, print the index for `<unk>` and the length of your resulting tagset. (0.5)

In [45]:
tag_numbers.setdefault("USR", len(tag_numbers))
tag_numbers.setdefault("HT", len(tag_numbers))
tag_numbers.setdefault("RT", len(tag_numbers))
tag_numbers.setdefault("URL", len(tag_numbers))
tag_numbers.setdefault("VPP", len(tag_numbers))
tag_numbers.setdefault("TD", len(tag_numbers))
tag_numbers.setdefault("O", len(tag_numbers))

word_numbers.setdefault("<unk>", len(word_numbers))
print word_numbers["<unk>"]
print len(tag_numbers)


26070
53


<b>Instructions</b>: now we can read the test tweets. Store them in the same format as the PTB corpora (list of lists containing (word, tag) index tuples). Do the same preprocessing steps that you did for the training tweets (lowercasing + replace special tokens). However, **do not** update the vocabulary. Why? Because the test set should simulate a real-world scenario, where out-of-vocabulary words can appear. Instead, after preprocessing each word, you should check if that word is in the vocabulary. If yes, just replace it with its index, otherwise you should replace it with the index for the `<unk>` token. Remember: you can reuse the code from the workshop for this task. Just be mindful that in the workshop we stored words and tags in two separate lists: here you should have a single list, as in the PTB corpus you preprocessed above.

When reading the POS tags for the test tweets you should do some additional preprocessing. There are three tags in this dataset which correspond to PTB tags but are represented with different names:
- "(". In PTB, this is represented as "-LRB-"
- ")". In PTB, this is represented as "-RRB-"
- "NONE". In PTB, this is represented as "-NONE-"

As you build the corpus for the test tweets, you should check if the tag for a word is one of the above. If yes, you should use the PTB equivalent instead. In practice, it is sufficient to ensure you use the correct index for the corresponding tag, using your tagset dictionary. This concept is sometimes referred as *tag harmonisation*, where two different tagsets are mapped to each other.

After this, print the first sentence of your preprocessed corpus. (0.5)

In [57]:
import urllib
try:
    urllib.request.urlretrieve("https://github.com/aritter/twitter_nlp/raw/master/data/annotated/pos.txt","pos.txt")
except: # Python 2
    urllib.urlretrieve("https://github.com/aritter/twitter_nlp/raw/master/data/annotated/pos.txt","pos.txt")

tweet_corpus = []
with open('pos.txt') as f:
    temps = set(["USR","RT","HT","URL"])
    tupList = []
    for line in f:
        if line.strip() == '':
            tweet_corpus.append(tupList)
            tupList = []
        else:
            word, pos = line.strip().split()
#             if word[0] =="@":
#                 word = "USER_TOKEN"
#             elif word[0] == "#":
#                 word = "HASHTAG_TOKEN"
#             elif word == "RT":
#                 word = "RETWEET_TOKEN"
#             elif word[:min(len("https://"),len(token))] == "https://":
#                 word = "URL_TOKEN"
#             elif word[:min(len("http://"),len(token))] == "http://":
#                 word = "URL_TOKEN"
#             else:
#                 word = word.lower() 
            
            if pos == "USR":
                word = "USER_TOKEN"
            elif pos == "HT":
                word = "HASHTAG_TOKEN"
            elif pos == "RT":
                word = "RETWEET_TOKEN"
            elif pos == "URL":
                word = "URL_TOKEN"
            else:
                word = word.lower()
            
            if pos == "(":
                pos = "-LRB-"
            elif pos == ")":
                pos = "-RRB-"
            elif pos == "NONE":
                pos = "-NONE-"

            if word in word_numbers:
                tupList.append((word_numbers[word],tag_numbers[pos]))
            else:
                tupList.append((word_numbers["<unk>"],tag_numbers[pos]))

print tweet_corpus[0]
    

[(11392, 46), (61, 19), (114, 11), (8, 7), (3224, 8), (170, 9), (325, 33), (1325, 19), (2375, 22), (3205, 12), (182, 9), (799, 2), (1522, 3), (16, 10), (8490, 0), (1146, 0), (2495, 0), (14038, 43), (26070, 0), (16, 10), (4263, 17), (1760, 4), (9464, 8), (2259, 17), (888, 4), (741, 8), (16, 10)]


<b>Hint</b>: if you did these steps correctly you should have 53 tags in your tagset and around 26000 words in your vocabulary.

### Part 2: Running the PTB tagger on the test tweets (1.5)

<b>Instructions</b>: your next task is to train a POS tagger on the PTB data and try it on the test tweets. This is exactly what we did in W7: feel free to reuse code. However, we are also gonna modify the code a bit.

Your first task is encapsulate the HMM training code into a function. You should name your function `count`. This function should take these input parameters:
- A tagged corpus, in the format described above (list of lists containing (word, tag) index tuples).
- The vocabulary (a dict).
- The tagset (a dict).

Output return values should contain:
- The initial tag probabilities (a vector).
- The transition probabilities (a matrix).
- The emission probabilities (a matrix).

Notice that in the workshop code the vocabulary and tagset were built as part of the training process. Here you should pass them explicitly as parameters instead. This is to ensure our tagger can take into account the words in the training tweets and the extra tags. Important: the workshop code initialise the probabilities with an `eps` value, to ensure you end up with non-zero probabilities for unseen events. You should do the same here.

After writing your function, run it on the PTB corpus to obtain the initial, transition and emission probabilities. (0.5)

In [58]:
import numpy as np
def count(tagged_tup_corpus,vocabulary,tagest): 
    S = len(tag_numbers)
    V = len(word_numbers)

    # initalise
    eps = 0.1
    pi = eps * np.ones(S)
    A = eps * np.ones((S, S))
    O = eps * np.ones((S, V))

    # count
    for sent in tagged_tup_corpus:
        last_tag = None
        for word, tag in sent:
            O[tag, word] += 1
            if last_tag == None:
                pi[tag] += 1
            else:
                A[last_tag, tag] += 1
            last_tag = tag
        
    # normalise
    pi /= np.sum(pi)
    for s in range(S):
        O[s,:] /= np.sum(O[s,:])
        A[s,:] /= np.sum(A[s,:])
    return pi,A,O



params = count(num_corpus,word_numbers,tag_numbers)
# print params


<b>Instructions</b>: now you should write a function for Viterbi. The input parameters are the same as in the workshop:
- The parameters (probabilities) of your HMM (a tuple (initial, transition, emission)).
- The input words (a list with numbers).

The output is slightly different though:
- A list of (word, tag) indices, containing the original input word and the predicted tag.

Run Viterbi on the test tweets and store the predictions in a list (might take a few seconds). Remember that in the processing part you stored the test tweets as (word, tag) indices lists: make sure your input to Viterbi are word index lists only. Print the first sentence of your predicted list. (0.5)

In [59]:
# def viterbi(params, observations):
#     pi, A, O = params
#     M = len(observations)
#     S = pi.shape[0]
    
#     alpha = np.zeros((M, S))
#     alpha[:,:] = float('-inf')
#     backpointers = np.zeros((M, S), 'int')
    
#     # base case
#     alpha[0, :] = pi * O[:,observations[0]]
    
#     # recursive case
#     for t in range(1, M):
#         for s2 in range(S):
#             for s1 in range(S):
#                 score = alpha[t-1, s1] * A[s1, s2] * O[s2, observations[t]]
#                 if score > alpha[t, s2]:
#                     alpha[t, s2] = score
#                     backpointers[t, s2] = s1
    
#     # now follow backpointers to resolve the state sequence
#     ss = []
#     ss.append((observations[M-1],np.argmax(alpha[M-1,:])))
#     for i in range(M-1, 0, -1):
#         ss.append((observations[i-1],backpointers[i, ss[-1][1]]))
        
#     return list(reversed(ss))



def viterbi(params, observations):
    pi, A, O = params
    M = len(observations)
    S = pi.shape[0]
    
    alpha = np.zeros((M, S))
    alpha[:,:] = float('-inf')
    backpointers = np.zeros((M, S), 'int')
    
    # base case
    alpha[0, :] = pi * O[:,observations[0]]
    
    # recursive case
    for t in range(1, M):
        temp = alpha[t-1] * A.T * O.T[observations[t]].reshape(S,1)
        for s2 in range(S):
            backpointers[t, s2] = np.argmax(temp[s2])
            alpha[t, s2] = temp[s2][backpointers[t, s2]]

    ss = []
    ss.append((observations[M-1],np.argmax(alpha[M-1,:])))
    for i in range(M-1, 0, -1):
        ss.append((observations[i-1],backpointers[i, ss[-1][1]]))
    return list(reversed(ss))



# predictionTupList = []
# for index,tupList in enumerate(tweet_corpus):
#     observations = [tup[0] for tup in tupList]
#     predictionTupList.append(viterbi(params, observations))
# #     if index == 10:
# #         break
#     print index

predictionTupList = []
for tupList in tweet_corpus:
    observations = [tup[0] for tup in tupList]
    predictionTupList.append(viterbi(params, observations))
print predictionTupList[0]

[(11392, 27), (61, 19), (114, 11), (8, 7), (3224, 8), (170, 9), (325, 33), (1325, 19), (2375, 22), (3205, 12), (182, 9), (799, 2), (1522, 3), (16, 10), (8490, 29), (1146, 8), (2495, 8), (14038, 10), (26070, 38), (16, 10), (4263, 29), (1760, 4), (9464, 8), (2259, 17), (888, 4), (741, 8), (16, 10)]


<b>Instructions</b>: you should now evaluate the results. Write a function that takes (word, tag) lists as inputs and outputs the tag sequence using the original tags in the tagset. Your inputs should be a sentence and the tag inverted index you built before.

Run this function on the predictions you obtained above **and** the test tweets, storing them in two separate lists. Finally, flat your predictions into a single list and do the same for the test tweets and report accuracy. (0.5)

In [60]:
def getTagSequenceList(tupSeqList):
    tagSeqList = []
    for tupSeq in tupSeqList:
        tagSeqList.append([tup[1] for tup in tupSeq])   
    return tagSeqList


from sklearn.metrics import accuracy_score as acc

all_test_tags = [tag for tags in getTagSequenceList(tweet_corpus) for tag in tags]
all_pred_tags = [tag for tags in getTagSequenceList(predictionTupList) for tag in tags]

print acc(all_test_tags, all_pred_tags)

0.6374053342113928


### Part 3: Adapting the tagger using prior information (1.5)

<b>Instructions</b>: now your task is to adapt the tagger using prior information. What do we mean by that? Remember from part 1 that the twitter tagset has some extra tags, related to special tokens such as mentions and hashtags. In other words, **we know beforehand** that these special tokens **should** have these tags. However, because these tags never appear in the PTB data, the tagger has no such information. We are going to add this in order to improve the tagger.

To recap, we know these things about the twitter data:
- username mentions should have the tag 'USR'
- hashtags should have the tag 'HT'
- retweet tokens should have the tag 'RT'
- URL tokens should have the tag 'URL'

Remember how we replace these tokens with unique special ones (such as 'USER_TOKEN')? Your task is to adapt the emission probabilities for these tokens. Modify the emission matrix: assign 1.0 probability for the emission P('USER_TOKEN'|'USR') and 0.0 for P(word|'USR') for all other words. Do the same for the other three special tags.

In order to do that, you should use the vocabulary and tagset dictionaries in order to obtain the indices for the corresponding words and tags. Then, use the indices to find the values in the emission matrix and modify them. Print your new emission matrix. (0.5)

In [61]:
O = params[2]
O[tag_numbers["USR"],:] = 0.0
O[tag_numbers["USR"],word_numbers["USER_TOKEN"]] = 1.0
O[tag_numbers["RT"],:] = 0.0
O[tag_numbers["RT"],word_numbers["RETWEET_TOKEN"]] = 1.0
O[tag_numbers["URL"],:] = 0.0
O[tag_numbers["URL"],word_numbers["URL_TOKEN"]] = 1.0
O[tag_numbers["HT"],:] = 0.0
O[tag_numbers["HT"],word_numbers["HASHTAG_TOKEN"]] = 1.0
print O





[[9.15362275e-05 1.74750980e-04 8.32147523e-06 ... 8.32147523e-06
  8.32147523e-06 8.32147523e-06]
 [1.33456113e-05 1.33456113e-05 6.51946457e-01 ... 1.33456113e-05
  1.33456113e-05 1.33456113e-05]
 [1.62519706e-05 1.62519706e-05 1.62519706e-05 ... 1.62519706e-05
  1.62519706e-05 1.62519706e-05]
 ...
 [3.83567949e-05 3.83567949e-05 3.83567949e-05 ... 3.83567949e-05
  3.83567949e-05 3.83567949e-05]
 [3.83567949e-05 3.83567949e-05 3.83567949e-05 ... 3.83567949e-05
  3.83567949e-05 3.83567949e-05]
 [3.83567949e-05 3.83567949e-05 3.83567949e-05 ... 3.83567949e-05
  3.83567949e-05 3.83567949e-05]]


<b>Instructions</b>: now evaluate your new tagger on the test tweets again. You should report accuracy but also do a fine-grained error analysis. Print the F-scores for **each tag**. <b>Hint:</b> use the "classification_report" function in scikit-learn for that. You should report the tags that performed the best and the worse. (0.5) 

In [62]:
from sklearn.metrics import classification_report

predictionTupList = []
# print len(tweet_corpus)
for tupList in tweet_corpus:
    observations = [tup[0] for tup in tupList]
    predictionTupList.append(viterbi(params, observations))
all_test_tags = [tag for tags in getTagSequenceList(tweet_corpus) for tag in tags]
all_pred_tags = [tag for tags in getTagSequenceList(predictionTupList) for tag in tags]

tag_name = {}
for key in tag_numbers.keys():
    tag_name[tag_numbers[key]] = key
print acc(all_test_tags, all_pred_tags)
all_test_tags_name = [tag_name[c] for c in all_test_tags]
all_pred_tags_name = [tag_name[c] for c in all_pred_tags]
print classification_report(all_test_tags_name,all_pred_tags_name)

print "NNP is the worst"
print "USR is the best"


0.6967402041488311
             precision    recall  f1-score   support

          #       0.00      0.00      0.00         0
          $       0.00      0.00      0.00         0
         ''       0.03      0.20      0.06        91
          ,       0.85      1.00      0.92       303
      -LRB-       0.00      0.00      0.00        32
     -NONE-       0.00      0.00      0.00         2
      -RRB-       0.05      0.15      0.07        34
          .       0.72      0.83      0.77       875
          :       0.97      0.76      0.85       562
         CC       0.96      0.88      0.92       305
         CD       0.59      0.59      0.59       268
         DT       0.74      0.93      0.82       825
         EX       0.38      0.80      0.52        10
         FW       0.00      0.00      0.00         3
         HT       1.00      0.99      0.99       135
         IN       0.82      0.88      0.85      1091
         JJ       0.63      0.59      0.61       670
        JJR       0.49    

<b>Instructions</b>: finally, based on the information you got above, do some analysis. Why do you think the tagger performed worse on the tags you mentioned above? How would you improve the tagger? Feel free to inspect some instances manually if you want (and show us if you do). Write your analysis in the markdown cell below. Notice that this question is inherently subjective: this is on purpose as you will be evaluated on your analytical abilities. But don't worry about going into depth: 2-4 sentences is enough (but feel free to write more if you need). (0.5)
    

<b>WRITE YOUR ANALYSIS HERE</b>
###my analysis
    Proper nouns (NNP) name specific people, places, things, or ideas. Since they these nouns are naming specific things, they always begin with a capital letter. But in our process word will be remain as lower. So lost some information. I think if we can keep the lower or upper character when it doesn't appear in the head. 


### Extra credits 1: Expectation-Maximisation (1.0)

<b>Instructions</b>: here your goal is to improve the tagger using **hard EM**. This question is divided in two parts. Because EM can take a long time to run we will modify our code above to make it faster and also more robust to underflow by making calculations in the log space.

Your first task is to modify the `count` and `viterbi` functions. For `count`, you should return log probabilities for all matrices. For `viterbi`, you should modify the code in the following way:
- Calculate scores using log probabilities. Remember that in log space, any products become sums. Also remember to make sure you change the base case as well (not only the inner loop).
- You should rewrite the algorithm in vectorised form, in order to make it more efficient. Remember that in Viterbi, the third (inner) `for` loop calculate scores for a single cell, while the second `for` loop calculates scores for a whole column. These operations can be made in parallel, which makes them amenable to vectorisation. Replace the second and third `for` loops in the code with appropriate vector and matrix operations. Remember to do the same to obtain the backpointers. <b>Hint</b>: start by vectorising the inner loop only and check if it is correct.

The second task is to implement EM. This can be done without the first step above but bear in mind that it will take much longer to run. Write a function that:

- 1) Tag a corpus using Viterbi and an initial tagger
- 2) Train a new tagger using the tagged corpora obtained above.
- 3) Repeat both steps above N times.

If you've done the main homework correctly you should be able to reuse the `count` and `viterbi` functions for this and the code should be very straightforward. You should pretrain your tagger using the steps in the main homework, including the tag adaptation in Part 3. Then, in the inner loop, use the training tweets as your unlabelled corpora. Run 5 iterations of EM and report test accuracy **at every iteration**.

EM can take a long time to train, even if you're using the vectorised Viterbi code. If it is too slow in your machine, you're allowed to use a subset of the training tweets for this task.

To get full marks, adapt the algorithm in the following manner:
- At step 2) above, when training the new tagger, combine **both** the PTB gold data and the tagged training tweets, instead of just using the training tweets.

This is an easy trick to obtain better results (and it is essentially **semi-supervised** learning).

In [35]:
import random
import collections

def viterbi(params, observations):
    pi, A, O = params
    M = len(observations)
    S = pi.shape[0]
    
    alpha = np.zeros((M, S))
    alpha[:,:] = float('-inf')
    backpointers = np.zeros((M, S), 'int')
    
    # base case
    alpha[0, :] = pi + O[:,observations[0]]
    
    # recursive case
    for t in range(1, M):
        temp = alpha[t-1] + A.T + O.T[observations[t]].reshape(S,1)
#         TODO
        for s2 in range(S):
            backpointers[t, s2] = np.argmax(temp[s2])
            alpha[t, s2] = temp[s2][backpointers[t, s2]]

    ss = []
    ss.append((observations[M-1],np.argmax(alpha[M-1,:])))
    for i in range(M-1, 0, -1):
        ss.append((observations[i-1],backpointers[i, ss[-1][1]]))
    return list(reversed(ss))
def count2(tagged_tup_corpus,vocabulary,tagset,O):
    S = len(tagset)
    V = len(vocabulary)

    # initalise
    eps = 0.1
    pi = eps * np.ones(S)
    A = eps * np.ones((S, S))
#     O = eps * np.ones((S, V))
    O = np.e**O
    
    PT = eps * np.ones(S)
    PW = eps * np.ones(V)
    PWT = eps * np.ones((V, S))
    CT = eps * np.ones(S)
    CTW = eps * np.ones((S, V))
    
    # count
    total = sum([len(sent) for sent in tagged_tup_corpus])
    for sent in tagged_tup_corpus:
        last_tag = None
        for word, tag in sent:
            PWT[word,tag] +=1
            PT[tag]+=1
            if last_tag == None:
                pi[tag] += 1
            else:
                A[last_tag, tag] += 1
            last_tag = tag
    
    PT /=np.sum(PT)
    for j in range(len(vocabulary)):
        PW[j] = np.sum(PT*O[:,j])
#     print O[:,10]
    PWT /= total
    
    #calculate p(t|w)
    for i in range(len(vocabulary)):
        PWT[i] /= PW[i]
    
    for sent in tagged_tup_corpus:
        last_tag = None
        for word, tag in sent:
            CT[tag]+=PWT[word,tag]
            CTW[tag,word]+=PWT[word,tag]
    for s in range(S):
        O[s] = CTW[s]/CT[s]
    
    
    # normalise
    pi /= np.sum(pi)
    for s in range(S):
        O[s,:] /= np.sum(O[s,:])
        A[s,:] /= np.sum(A[s,:])
    pi = np.log(pi)
    O = np.log(O)
    A = np.log(A)
    return pi,A,O

def count(tagged_tup_corpus,vocabulary,tagset): 
    S = len(tagset)
    V = len(vocabulary)

    # initalise
    eps = 0.1
    pi = eps * np.ones(S)
    A = eps * np.ones((S, S))
    O = eps * np.ones((S, V))

    # count
    for sent in tagged_tup_corpus:
        last_tag = None
        for word, tag in sent:
            O[tag, word] += 1
            if last_tag == None:
                pi[tag] += 1
            else:
                A[last_tag, tag] += 1
            last_tag = tag
        
    # normalise
    pi /= np.sum(pi)
    for s in range(S):
        O[s,:] /= np.sum(O[s,:])
        A[s,:] /= np.sum(A[s,:])
    pi = np.log(pi)
    O = np.log(O)
    A = np.log(A)
    return pi,A,O
params = count(num_corpus,word_numbers,tag_numbers)

O = params[2]
O[tag_numbers["USR"],:] = float('-inf')
O[tag_numbers["USR"],word_numbers["USER_TOKEN"]] = 0
O[tag_numbers["RT"],:] = float('-inf')
O[tag_numbers["RT"],word_numbers["RETWEET_TOKEN"]] = 0
O[tag_numbers["URL"],:] = float('-inf')
O[tag_numbers["URL"],word_numbers["URL_TOKEN"]] = 0
O[tag_numbers["HT"],:] = float('-inf')
O[tag_numbers["HT"],word_numbers["HASHTAG_TOKEN"]] = 0

predictionTupList = []
for tupList in tweet_corpus:
    observations = [tup[0] for tup in tupList]
    predictionTupList.append(viterbi(params, observations))
all_test_tags = [tag for tags in getTagSequenceList(tweet_corpus) for tag in tags]
    # for predictions, we need to obtain the original tag from the index
all_pred_tags = [tag for tags in getTagSequenceList(predictionTupList) for tag in tags]

# print acc(all_test_tags, all_pred_tags)
print "---------version 1--------"
for i in range(5):
#     params = count2(predictionTupList+num_corpus,word_numbers,tag_numbers,params[2])
    params = count(predictionTupList+num_corpus,word_numbers,tag_numbers)
    predictionTupList = []
    O = params[2]
    O[tag_numbers["USR"],:] = float('-inf')
    O[tag_numbers["USR"],word_numbers["USER_TOKEN"]] = 0
    O[tag_numbers["RT"],:] = float('-inf')
    O[tag_numbers["RT"],word_numbers["RETWEET_TOKEN"]] = 0
    O[tag_numbers["URL"],:] = float('-inf')
    O[tag_numbers["URL"],word_numbers["URL_TOKEN"]] = 0
    O[tag_numbers["HT"],:] = float('-inf')
    O[tag_numbers["HT"],word_numbers["HASHTAG_TOKEN"]] = 0
    for tupList in tweet_corpus:
        observations = [tup[0] for tup in tupList]
        predictionTupList.append(viterbi(params, observations))
#     all_test_tags = [tag for tags in getTagSequenceList(tweet_corpus) for tag in tags]
    # for predictions, we need to obtain the original tag from the index
    all_pred_tags = [tag for tags in getTagSequenceList(predictionTupList) for tag in tags]

    print acc(all_test_tags, all_pred_tags)

print "----------version 2----------"
print "I use the last params and tag together to update the new params"
params = count(num_corpus,word_numbers,tag_numbers)

O = params[2]
O[tag_numbers["USR"],:] = float('-inf')
O[tag_numbers["USR"],word_numbers["USER_TOKEN"]] = 0
O[tag_numbers["RT"],:] = float('-inf')
O[tag_numbers["RT"],word_numbers["RETWEET_TOKEN"]] = 0
O[tag_numbers["URL"],:] = float('-inf')
O[tag_numbers["URL"],word_numbers["URL_TOKEN"]] = 0
O[tag_numbers["HT"],:] = float('-inf')
O[tag_numbers["HT"],word_numbers["HASHTAG_TOKEN"]] = 0

predictionTupList = []
for tupList in tweet_corpus:
    observations = [tup[0] for tup in tupList]
    predictionTupList.append(viterbi(params, observations))
all_test_tags = [tag for tags in getTagSequenceList(tweet_corpus) for tag in tags]
    # for predictions, we need to obtain the original tag from the index
all_pred_tags = [tag for tags in getTagSequenceList(predictionTupList) for tag in tags]

for i in range(5):
    params = count2(predictionTupList+num_corpus,word_numbers,tag_numbers,params[2])
    predictionTupList = []
    O = params[2]
    O[tag_numbers["USR"],:] = float('-inf')
    O[tag_numbers["USR"],word_numbers["USER_TOKEN"]] = 0
    O[tag_numbers["RT"],:] = float('-inf')
    O[tag_numbers["RT"],word_numbers["RETWEET_TOKEN"]] = 0
    O[tag_numbers["URL"],:] = float('-inf')
    O[tag_numbers["URL"],word_numbers["URL_TOKEN"]] = 0
    O[tag_numbers["HT"],:] = float('-inf')
    O[tag_numbers["HT"],word_numbers["HASHTAG_TOKEN"]] = 0
    for tupList in tweet_corpus:
        observations = [tup[0] for tup in tupList]
        predictionTupList.append(viterbi(params, observations))
#     all_test_tags = [tag for tags in getTagSequenceList(tweet_corpus) for tag in tags]
    # for predictions, we need to obtain the original tag from the index
    all_pred_tags = [tag for tags in getTagSequenceList(predictionTupList) for tag in tags]

    print acc(all_test_tags, all_pred_tags)



---------version 1--------
0.699703654922621
0.6987816924596641
0.6979255844583471
0.6966743496871913
0.6964767863022719
----------version 2----------
I use the last params and tag together to update the new params
0.7183404675666777
0.7218307540335858
0.7237405334211393
0.7234112611129404
0.7230819888047415


### Extra credits 2: Soft EM using Forward-Backward (1.0)

<b>Instructions</b>: This is only for the truly intrepid: expect a substantial amount of work to get full marks in this. The goal is to perform **soft EM** using the Forward-Backward algorithm and expected counts. You will need to implement and adapt a set of functions for this task:
- The `count` method, which will still be used for pretraining, needs to also calculate **final** probabilities and return these as part of the output. These are never used in Viterbi but are essential for Forward-Backward.
- Implement the `forward` function. If you want to work in log space you should be careful because it requires summing probabilities. Hint: check the Scipy function `logsumexp` for that. Remember: the algorithm is very similar to Viterbi so feel free to use it as a starting point. The function should return the matrix with the alpha values and the marginal probability for the sentence.
- Implement the `backward` function. This should return a matrix with the beta values and the marginal. Remember the very useful sanity check: for the same sentence and tagger, the marginals returned from `forward` and `backward` should match.
- Implement the `expected_count` function, which is similar to `count` in the sense that it trains a tagger and output new parameters. However, it should have the alpha and beta matrices as inputs, as well as the marginals.

After implementing all of this, rerun EM using the soft approach and evaluate it in a similar way as done in Extra credits 1.


In [36]:
params = count(num_corpus,word_numbers,tag_numbers)

O = params[2]
O[tag_numbers["USR"],:] = float('-inf')
O[tag_numbers["USR"],word_numbers["USER_TOKEN"]] = 0
O[tag_numbers["RT"],:] = float('-inf')
O[tag_numbers["RT"],word_numbers["RETWEET_TOKEN"]] = 0
O[tag_numbers["URL"],:] = float('-inf')
O[tag_numbers["URL"],word_numbers["URL_TOKEN"]] = 0
O[tag_numbers["HT"],:] = float('-inf')
O[tag_numbers["HT"],word_numbers["HASHTAG_TOKEN"]] = 0

predictionTupList = []
for tupList in tweet_corpus:
    observations = [tup[0] for tup in tupList]
    predictionTupList.append(viterbi(params, observations))
# print predictionTupList[0]
all_test_tags = [tag for tags in getTagSequenceList(tweet_corpus) for tag in tags]
    # for predictions, we need to obtain the original tag from the index
all_pred_tags = [tag for tags in getTagSequenceList(predictionTupList) for tag in tags]

# print acc(all_test_tags, all_pred_tags)

def count3(tagged_tup_corpus,vocabulary,tagset): 
    S = len(tagset)
    V = len(vocabulary)

    # initalise
    eps = 0.1
    pi = eps * np.ones(S)
    pe = eps * np.ones(S)
    A = eps * np.ones((S, S))
    O = eps * np.ones((S, V))
    B = eps * np.ones((V, S))

    # count
    for sent in tagged_tup_corpus:
        last_tag = None
        for word, tag in sent:
            O[tag, word] += 1
            B[word, tag] +=1
            if last_tag == None:
                pi[tag] += 1
            else:
                A[last_tag, tag] += 1
            last_tag = tag
        pe[last_tag]+=1
    # normalise
    pi /= np.sum(pi)
    for s in range(S):
        O[s,:] /= np.sum(O[s,:])
        A[s,:] /= np.sum(A[s,:])
    for v in range(V):
        B[v,:] /= np.sum(B[v,:])
#     pi = np.log(pi)
#     O = np.log(O)
#     A = np.log(A)
    return pi,pe,A,O,B
    

def forward(params,M,S):
    pi = params[0]
    pe = params[1]
    A = params[2]
    O = params[3]
    B = params[4]
    alpha = np.zeros((M, S))
    alpha[0, :] = pi * O[:,observations[0]]
    # recursive case
    for t in range(1, M):
        temp = alpha[t-1] * A.T * O.T[observations[t]].reshape(S,1)
        alpha[t] = np.matmul(temp,[1]*S)
    #TODO add <end> to A
    #which one to choose
#     total = np.sum(alpha[M-1,:] + A[:,S]) 
    total = np.sum(alpha[M-1,:] * pe)
    return total, alpha
    
def backforward(params,M,S):
    pi = params[0]
    pe = params[1]
    A = params[2]
    O = params[3]
    B = params[4]
    beta = np.zeros((M, S))
    #TODO
#     for t in range(S):
        
#         beta[M, t] = A[:,<end>]
#         beta[M-1, t] = A[:,S]
        #in order to setvalues not address
    beta[M-1] = pe[:]
    for i in range(M-2, -1, -1):
        temp = beta[i+1] * A * O[:,observations[i+1]]
        beta[i] = np.matmul(temp,[1]*S) 
    total = np.sum(beta[0,:] * pi * O[:,observations[0]]) 
    return total, beta

def expected_count(alpha,totalF,beta,totalB,observations,CT,CWT,CTT,CT2,params,S,V):
    M = len(observations)
    
    pi = params[0]
    pe = params[1]
    A = params[2]
    O = params[3]
    B = params[4]
    
    # count
    for i in range(len(observations)):
        CT += (alpha[i] * beta[i] /totalF)
        CWT[observations[i]] += (alpha[i] * beta[i] /totalF)
        if i<len(observations)-1:
            temp = alpha[i] * A.T * O[:,observations[i+1]].reshape(S,1)*beta[i+1].reshape(S,1)
            CTT +=temp
            CT2 += np.matmul([1]*S,temp)
            
#             for s1 in range(S):
#                 temp = alpha[i] * A[:,s1]*O[s1,observations[i+1]]*beta[i+1,s1]/totalF
#                 CTT[s1]+=temp
#                 CT += temp
                
#                 for s2 in range(S):
#                     temp = alpha[i,s2]*A[s2,s1]*O[s1,observations[i+1]]*beta[i+1,s1]/totalF
#                     CTT[s1,s2] += temp
#                     CT[s2] += temp
        
        #TODO when at end
#         for s in range(S):
#             CT[s] += alpha[i,s] * beta[i,s] / totalF
#             CTW[observations[i],s] += (alpha[i,s] * beta[i,s] /totalF)
        
        
#         pe[last_tag]+=1
    # normalise
    pi /= np.sum(pi)
    for s in range(S):
        O[s,:] /= np.sum(O[s,:])
        A[s,:] /= np.sum(A[s,:])
    
    return CT,CWT,CTT,CT2


# pi = eps * np.ones(S)
# pe = eps * np.ones(S)
# A = eps * np.ones((S, S))
# O = eps * np.ones((S, V))
V = len(word_numbers)
S = len(tag_numbers)
for i in range(5):
    eps = 0.1
    params = count3(predictionTupList+num_corpus,word_numbers,tag_numbers)
    
    CT = eps * np.ones(S)
    CWT = eps * np.ones((V, S))
    CTT = eps * np.ones((S, S))
    CT2 = eps * np.ones(S)
#     print len(tweet_corpus)
#     tcount = 0
    for tupList in tweet_corpus:
        M = len(tupList)
        observations = [tup[0] for tup in tupList]
        totalF,alpha = forward(params,M,S)
        totalB,beta = backforward(params,M,S)
        expected_count(alpha,totalF,beta,totalB,observations,CT,CWT,CTT,CT2,params,S,V)
#         tcount+=1
#         print tcount
    O = (CWT/CT).T
    A = (CTT/CT2).T
    pi = params[0]
    
    for s in range(S):
        O[s,:] /= np.sum(O[s,:])
        A[s,:] /= np.sum(A[s,:])
    
    pi = np.log(pi)
    O = np.log(O)
    A = np.log(A)
    
    
    params = [pi, A, O]
    
    predictionTupList = []
    for tupList in tweet_corpus:
        observations = [tup[0] for tup in tupList]
        predictionTupList.append(viterbi(params, observations))
    all_test_tags = [tag for tags in getTagSequenceList(tweet_corpus) for tag in tags]
    # for predictions, we need to obtain the original tag from the index
    all_pred_tags = [tag for tags in getTagSequenceList(predictionTupList) for tag in tags]
    print acc(all_test_tags, all_pred_tags)
    



0.7072769180111953
0.7077378992426737
0.7078037537043135
0.7078037537043135
0.7078037537043135
